# Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
import numpy as np

# Database creations using pytorch Dataset 

In [2]:
class ImageQualityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(os.getcwd(), self.data.iloc[idx, 3])  # image_path column
        image = Image.open(img_name).convert('RGB')
        quality = self.data.iloc[idx, 0]  # Quality column
        authenticity = self.data.iloc[idx, 1]  # Authenticity column
        labels = torch.tensor([quality, authenticity], dtype=torch.float)


        if self.transform:
            image = self.transform(image)

        return image, labels


# Definitions of the models

In [3]:
class QualityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Predict quality and realness
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features

## Setup section

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations for the ImageNet dataset
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

annotations_file = 'Dataset/AIGCIQA2023/mos_data.csv'

# Create the dataset
dataset = ImageQualityDataset(csv_file=annotations_file, transform=data_transforms)

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


# Create data loaders
BATCH_SIZE = 64
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Create a dictionary containing the data loaders
dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader,
    'test': test_dataloader
}



# Models loading

In [5]:
BASELINE_MODEL_PATH = 'Models/VGG-16_finetuned_regression.pth'
NOISY_PRUNED_MODEL_PATH = 'Models/noise_out_pruned_model.pth'
NEGATIVE_IMPACT_PRUNED_MODEL_PATH = 'Models/negative_impact_pruned_model.pth'

noisy_pruned_model = QualityPredictor()
noisy_pruned_model.load_state_dict(torch.load(NOISY_PRUNED_MODEL_PATH, weights_only=True))

negative_impact_pruned_model = QualityPredictor()
negative_impact_pruned_model.load_state_dict(torch.load(NEGATIVE_IMPACT_PRUNED_MODEL_PATH,weights_only=True))

baseline_model = QualityPredictor()
baseline_model.load_state_dict(torch.load(BASELINE_MODEL_PATH,weights_only=True))

<All keys matched successfully>

# Next steps

- Getting the activations of each channel in the last convolutional layer for your images 
- Weighting each channel's activation map by its computed importance score
- Aggregating these weighted activations to form a final heatmap
- Visualizing this heatmap overlaid on the original image

# 1) Sort the importance scores by channel ID and normalize them to sum to 1

In [41]:
#set numpy print options
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
importance_scores = np.load('Ranking_arrays/importance_scores.npy')
importance_scores_sorted_by_index = importance_scores[importance_scores[:,0].argsort()][:,1]

from scipy.special import softmax

# Normalize the importance scores using the softmax function to get the normalized importance scores (importance scores that sum to 1)
normalized_importance_scores_sorted_by_index = softmax(importance_scores_sorted_by_index)


# 2) Extracting the activations of the last convolutional layer for the images in the dataset

In [65]:

class FeatureMapHook:
    """Hook to extract feature maps from neural network layers."""
    
    def __init__(self):
        self.feature_maps = []
    
    def __call__(self, module, input, output):
        # Detach from computation graph and move to CPU
        self.feature_maps.append(output.detach().cpu())

def get_feature_maps(model, dataloader, layer_name, device):
    """
    Extracts the feature maps of a specific layer from a model.
    
    Args:
        model (nn.Module): The neural network model.
        dataloader (DataLoader): DataLoader for evaluation.
        layer_name (str): The name of the layer to extract feature maps from.
        device (str): Device to run the model on ('cuda' or 'cpu').
        
    Returns:
        np.ndarray: The feature maps as a numpy array with shape (240, num_features).
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)
    
    # Register a hook to extract feature maps
    hook = FeatureMapHook()
    target_layer = dict(model.named_modules())[layer_name]
    hook_handle = target_layer.register_forward_hook(hook)
    
    # Forward pass to extract feature maps from the dataloader
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            model(inputs)

    # Remove the hook
    hook_handle.remove()
    
    # Process the feature maps to get the desired shape
    all_features = []
    
    for batch_features in hook.feature_maps:
        # Add batch features to our collection
        all_features.append(batch_features)
    
    # Concatenate all batches and convert to numpy
    features_tensor = torch.cat(all_features, dim=0)
    
    # Ensure we have exactly the number of samples we expect in the dataloader 
    assert features_tensor.shape[0] == len(dataloader.dataset) 
    
    # Convert to numpy array
    features_array = features_tensor.numpy()
    
    return features_array

test_functions_dataloader = DataLoader(torch.utils.data.Subset(val_dataset, range(1)), 
                                     batch_size=1, 
                                     shuffle=True, 
                                     num_workers=0)

layer_28_feature_maps_base = get_feature_maps(baseline_model, test_functions_dataloader, 'features.28', device)
layer_28_feature_maps_noisy = get_feature_maps(noisy_pruned_model, test_functions_dataloader, 'features.28', device)
layer_28_feature_maps_neg_impact = get_feature_maps(negative_impact_pruned_model, test_functions_dataloader, 'features.28', device)


# layer_28_feature_maps_base = get_feature_maps(baseline_model, dataloaders['val'], 'features.28', device)
# layer_28_feature_maps_noisy = get_feature_maps(noisy_pruned_model, dataloaders['val'], 'features.28', device)
# layer_28_feature_maps_neg_impact = get_feature_maps(negative_impact_pruned_model, dataloaders['val'], 'features.28', device)

# check shapes
print("Layer 28 feature maps shape (Baseline): ", layer_28_feature_maps_base.shape)
print("Layer 28 feature maps shape (Noisy Pruned): ", layer_28_feature_maps_noisy.shape)
print("Layer 28 feature maps shape (Negative Impact Pruned): ", layer_28_feature_maps_neg_impact.shape)



Layer 28 feature maps shape (Baseline):  (1, 512, 14, 14)
Layer 28 feature maps shape (Noisy Pruned):  (1, 512, 14, 14)
Layer 28 feature maps shape (Negative Impact Pruned):  (1, 512, 14, 14)


In [ ]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm
import pathlib
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

def compute_heatmap(feature_maps, scores):
    """
    Computes a weighted average of a set of feature maps using a set of corresponding scores.

    Args:
    - feature_maps (numpy.ndarray): A 4D array of feature maps (shape 144 x 512 x 14 x 14).
    - scores (numpy.ndarray): A 2D array of scores for each feature map (shape 144 x 512).

    Returns:
    numpy.ndarray: A 3D array representing the computed heatmap (shape 144 x 14 x 14).
    """
    return np.average(feature_maps, weights=scores, axis=0)

def compute_heatmap_normalized(feature_maps, scores):
    # Calculate the minimum and maximum values along the (1, 2) axes
    min_vals = np.min(feature_maps, axis=(1, 2), keepdims=True)
    max_vals = np.max(feature_maps, axis=(1, 2), keepdims=True)

    # Handle division by zero by replacing zero denominators with a small epsilon value
    eps = 1e-10
    denom = max_vals - min_vals
    denom[denom == 0] = eps

    # Normalize the feature maps within each 14x14 map
    normalized_feature_maps = (feature_maps - min_vals) / denom

    # Handle NaN values by replacing them with 0
    normalized_feature_maps = np.nan_to_num(normalized_feature_maps)

    # Compute the weighted average of the normalized feature maps using the scores
    weighted_average = np.average(normalized_feature_maps, weights=scores, axis=0)

    return weighted_average

def overlay_mask(img: Image.Image, mask: np.array, colormap: str = "jet_r", alpha: float = 0.7) -> Image.Image:
    """Overlay a colormapped mask on a background image
    Args:
        img: background image
        mask: mask to be overlayed in grayscale
        colormap: colormap to be applied on the mask
        alpha: transparency of the background image
    Returns:
        overlayed image
    """
    
    if not isinstance(alpha, float) or alpha < 0 or alpha >= 1:
        raise ValueError("alpha argument is expected to be of type float between 0 and 1")

    # Get the specified colormap
    cmap = cm.get_cmap(colormap)
    
    # Resize mask to match the image dimensions
    overlay = cv2.resize(mask, img.size)
    
    # Normalize the mask values to range [0,1]
    overlay = overlay / overlay.max()
    
    # Apply colormap to the mask and convert to RGB
    # The ** 2 applies a quadratic transformation to enhance contrast
    overlay = (255 * cmap(np.asarray(overlay) ** 2)[:, :, :3]).astype(np.uint8)
    
    # Blend the original image with the colored mask
    # alpha controls how much of the original image shows through
    overlayed_img = Image.fromarray((alpha * np.asarray(img) + (1 - alpha) * np.asarray(overlay)).astype(np.uint8))

    return overlayed_img

def create_heatmaps_from_dataloader(dataloader, batch_feature_maps,channel_importance_scores, output_dir=None):
    """
    Create and save heatmaps for images in a dataloader using channel importance scores.
    
    Args:
        model: The model to extract feature maps
        dataloader: PyTorch DataLoader containing images
        channel_importance_scores: Array of importance scores for each channel (512,)
        output_dir: Directory to save the output heatmaps
        device: Device to run model on ('cuda' or 'cpu')
        feature_layer_name: Name of the layer to extract features from
        batch_limit: Maximum number of batches to process (None = all)
    """
    # Create output directory if it doesn't exist
    output_dir = pathlib.Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
        

    # Define inverse normalization to recover original images
    inv_normalize = transforms.Compose([
        transforms.Normalize(
            mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
            std=[1/0.229, 1/0.224, 1/0.225]
        )
    ])
    
    # Process each batch
    heatmap_list = []
    img_idx = 0
    
    for batch_idx, (inputs, labels) in enumerate(dataloader):
        # Process each image in the batch
        for i in range(inputs.size(0)):
            # Get feature maps for this image
            feature_maps = batch_feature_maps[i]  # Shape: (512, H, W)
            
            # Compute heatmap
            heatmap = compute_heatmap_normalized(feature_maps, channel_importance_scores)
            heatmap_list.append(heatmap)
            
            # Convert tensor back to image
            img_tensor = inputs[i].cpu()
            # Denormalize the image
            img_tensor = inv_normalize(img_tensor)
            img_tensor = torch.clamp(img_tensor, 0, 1)
            img_np = img_tensor.permute(1, 2, 0).numpy() * 255
            original_img = Image.fromarray(img_np.astype(np.uint8))
            
            # Overlay heatmap on original image
            overlaid_img = overlay_mask(original_img, heatmap)
            
            # Save the overlaid image
            output_path = output_dir / f"heatmap_idx{img_idx}.png"
            overlaid_img.save(output_path)
            
            print(f"Processed image {img_idx}")
            img_idx += 1
        
        print(f"Completed batch {batch_idx+1}/{len(dataloader)}")

    return heatmap_list

In [ ]:
# Get just the first 5 elements of the validation dataset to test the functions above

OUTPUT_DIR_BASE = 'Heatmaps_images/base_model'
OUTPUT_DIR_NOISY = 'Heatmaps_images/noisy_pruned_model'
OUTPUT_DIR_NEG_IMPACT = 'Heatmaps_images/negative_impact_pruned_model'

batch_features_base = layer_28_feature_maps_base
batch_features_noisy = layer_28_feature_maps_noisy
channel_importance_scores = normalized_importance_scores_sorted_by_index

heatmap_list_base = create_heatmaps_from_dataloader(test_functions_dataloader, batch_features_base, channel_importance_scores, OUTPUT_DIR_BASE)

heatmap_list_noisy = create_heatmaps_from_dataloader(test_functions_dataloader, batch_features_noisy, channel_importance_scores, OUTPUT_DIR_NOISY)

heatmap_list_neg_impact = create_heatmaps_from_dataloader(test_functions_dataloader, layer_28_feature_maps_neg_impact, channel_importance_scores, OUTPUT_DIR_NEG_IMPACT)



Processed image 0
Completed batch 1/1
Processed image 0
Completed batch 1/1
Processed image 0
Completed batch 1/1


/tmp/ipykernel_399399/1726644971.py:60: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(colormap)
